In [1]:
import nltk
#nltk.download()
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer,RegexpStemmer
from nltk.tokenize import RegexpTokenizer,word_tokenize,sent_tokenize
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.pipeline import Pipeline

In [2]:
reviews=pd.read_csv('Amazon_Reviews.csv')
reviews.head(10)

,Review,Label
0,Stuning even for the non-gamer: This sound tr...,1
1,The best soundtrack ever to anything.: I'm re...,1
2,Amazing!: This soundtrack is my favorite musi...,1
3,Excellent Soundtrack: I truly like this sound...,1
4,"Remember, Pull Your Jaw Off The Floor After H...",1
5,an absolute masterpiece: I am quite sure any ...,1
6,"Buyer beware: This is a self-published book, ...",0
7,Glorious story: I loved Whisper of the wicked...,1
8,A FIVE STAR BOOK: I just finished reading Whi...,1
9,Whispers of the Wicked Saints: This was a eas...,1


In [3]:
reviews['Label'].value_counts()

1    111
0     88
Name: Label, dtype: int64

In [4]:
def preprocessing(review):
    final_token=' '
    tokenizer = RegexpTokenizer(r"\w+")
     
    tokens=tokenizer.tokenize(review)
    eng_stop=stopwords.words('english')
    new_tokens=[token for token in tokens if token.lower() not in eng_stop]
    lem=WordNetLemmatizer()
    norm_token=[ lem.lemmatize(token,pos='v') for token in new_tokens]
    return final_token.join(norm_token)

    

In [5]:
reviews['token']=reviews['Review'].apply(lambda x:preprocessing(x))

In [6]:
reviews.head(10)

,Review,Label,token
0,Stuning even for the non-gamer: This sound tr...,1,Stuning even non gamer sound track beautiful p...
1,The best soundtrack ever to anything.: I'm re...,1,best soundtrack ever anything read lot review ...
2,Amazing!: This soundtrack is my favorite musi...,1,Amazing soundtrack favorite music time hand in...
3,Excellent Soundtrack: I truly like this sound...,1,Excellent Soundtrack truly like soundtrack enj...
4,"Remember, Pull Your Jaw Off The Floor After H...",1,Remember Pull Jaw Floor Hearing play game know...
5,an absolute masterpiece: I am quite sure any ...,1,absolute masterpiece quite sure actually take ...
6,"Buyer beware: This is a self-published book, ...",0,Buyer beware self publish book want know read ...
7,Glorious story: I loved Whisper of the wicked...,1,Glorious story love Whisper wicked saint story...
8,A FIVE STAR BOOK: I just finished reading Whi...,1,FIVE STAR BOOK finish read Whisper Wicked sain...
9,Whispers of the Wicked Saints: This was a eas...,1,Whispers Wicked Saints easy read book make wan...


In [7]:

data=reviews['token']
label=reviews['Label']

In [8]:
data.head(1)

0    Stuning even non gamer sound track beautiful p...
Name: token, dtype: object

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data,label,test_size=.15,random_state=42)

In [10]:
tfidf=TfidfVectorizer()
x_train1=tfidf.fit_transform(x_train)
x_test1=tfidf.transform(x_test)

x_train1.toarray()[:5]

array([[0.        , 0.10689724, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.08848379, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [13]:
GNB=GaussianNB()
GNB.fit(x_train1.toarray(),y_train)

GaussianNB()

In [14]:
pred=GNB.predict(x_test1.toarray())

In [15]:
df=pd.DataFrame({'review':[tfidf.inverse_transform(i) for i in x_test1.toarray()],'predicrion':pred})
df.head()

,review,predicrion
0,"[[always, best, big, boys, cd, come, ever, eve...",1
1,"[[author, book, feature, fool, haddon, make, m...",0
2,"[[animals, animation, cute, design, different,...",1
3,"[[achieve, also, beyond, family, fascinate, go...",0
4,"[[amazon, back, bad, bother, build, buy, colle...",0


In [16]:
from sklearn.metrics import classification_report
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

           0       0.64      0.70      0.67        10
           1       0.84      0.80      0.82        20

    accuracy                           0.77        30
   macro avg       0.74      0.75      0.74        30
weighted avg       0.77      0.77      0.77        30



In [ ]:
sen=input()
new_review=pd.Series({'review':sen})

In [24]:
import spacy
eng_words=spacy.load('en_core_web_sm')
doc=eng_words(sen)
from spacy import displacy
displacy.render(doc,style='ent',jupyter=True)


In [ ]:
token_sen=new_review.apply(preprocessing)
token_sen

In [ ]:
dtm=tfidf.transform(token_sen)
sentiment=GNB.predict(dtm)

if sentiment==0:
    print('negative')
else:
    print('positive')